<a href="https://colab.research.google.com/github/bishrfaz/ClinicalBert/blob/master/Time%20Series%20Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TIME SERIES DATA FORECASTING FAVOURITA DATASET**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing necessary libraries


In [ ]:
# Data Handling
from dotenv import dotenv_values
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

# Feature Processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Modelling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import pickle


# Other Packages
import warnings

warnings.filterwarnings("ignore")

Getting Familarized with the Datasets

In [ ]:
#function to display the details of datasets
def details(dataset,datasetname):
  print(f"dimension of {datasetname} {dataset.shape}")
  print("="*50)
  print()
  print(dataset.head())
  print("="*50)
  print()
  print(dataset.info())
  print("="*50)
  print()
  print(dataset.describe())
  print("="*50)
  print("Missing Values")
  print(dataset.isnull().sum())
  print("="*50)
  print()



Detail of oil.csv

In [ ]:
#importing oil dataset from drive and displaying the details
oil_df=pd.read_csv("/content/drive/MyDrive/data/oil.csv")
details(oil_df,"oil.csv")


Details of holiday events.csv

In [ ]:
holiday_df=pd.read_csv("/content/drive/MyDrive/data/holidays_events.csv")
details(holiday_df,"holiday_events.csv")


Details of stores.csv


In [ ]:
stores_df=pd.read_csv("/content/drive/MyDrive/data/stores.csv")
details(stores_df,"stores.csv")


Details of transction.csv


In [ ]:
transactions_df=pd.read_csv("/content/drive/MyDrive/data/transactions.csv")
details(transactions_df,"transactions.csv")


Details of train.csv

In [ ]:
train_df=pd.read_csv("/content/drive/MyDrive/data/train.csv")
details(train_df,"train.csv")


Details of the test.csv

In [ ]:
test_df=pd.read_csv("/content/drive/MyDrive/data/test.csv")
details(test_df,"test.csv")


By looking upon the details we came to know that the datatype of date is object for exploitation the datatype should be changed to datetime


There are 43 missing values in oil price in oil.csv

Transforming Object to Datetime

In [ ]:
def datetimetransform(dataset):
  dataset["date"]=pd.to_datetime(dataset["date"])
datetimetransform(oil_df)
datetimetransform(holiday_df)
datetimetransform(transactions_df)
datetimetransform(train_df)
datetimetransform(test_df)

Handling missing values

In [ ]:
#visualize the oil.csv to see how large the gaps are
fig=px.line(oil_df,x='date',y="dcoilwtico")
fig.update_layout(title="Trend of oil prices",xaxis_title="Date",yaxis_title='oil price')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Backward fill is suitable as there are irregular gaps in  the plot an its the most suitable way as the future values are the best estimator for filling the gap

In [ ]:
oil_df['dcoilwtico']=oil_df['dcoilwtico'].fillna(method='bfill')

Cross checking for missing values in oil.csv

In [ ]:
details(oil_df,"oil.csv")

Check the continouity of the Date in the test.csv

In [ ]:
min_date = train_df['date'].min()
max_date = train_df['date'].max()
expected_dates = pd.date_range(start=min_date, end=max_date)

missing_dates = expected_dates[~expected_dates.isin(train_df['date'])]

if len(missing_dates) == 0:
    print("The train dataset is complete. It includes all the required dates.")
else:
    print("The train dataset is incomplete. The following dates are missing:")
    print(missing_dates)


handle the missing dates

In [ ]:
# Complete the missing dates in the train dataset
# Create an index of the missing dates as a DatetimeIndex object
missing_dates = pd.Index(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]')

# Create a DataFrame with the missing dates, using the 'date' column
missing_data = pd.DataFrame({'date': missing_dates})

# Concatenate the original train dataset and the missing data DataFrame
# ignore_index=True ensures a new index is assigned to the resulting DataFrame
train_df = pd.concat([train_df, missing_data], ignore_index=True)

# Sort the DataFrame based on the 'date' column in ascending order
train_df.sort_values('date', inplace=True)

confirming the dataset


In [ ]:
min_date = train_df['date'].min()
max_date = train_df['date'].max()
expected_dates = pd.date_range(start=min_date, end=max_date)

missing_dates = expected_dates[~expected_dates.isin(train_df['date'])]

if len(missing_dates) == 0:
    print("The train dataset is complete. It includes all the required dates.")
else:
    print("The train dataset is incomplete. The following dates are missing:")
    print(missing_dates)


Merge the oil.csv,holiday_events.csv,tranaction.csv to the train.csv using inner join on date coloumn

merge store.csv using store_nbr column

In [ ]:
# Merging the common columns ('store_nbr' and 'date') in the datasets using the inner merge() function
# Merge train_df with stores_df based on 'store_nbr' column
merged_df1 = train_df.merge(stores_df, on='store_nbr', how='inner')

# Merge merged_df1 with transactions_df based on 'date' and 'store_nbr' columns
merged_df2 = merged_df1.merge(transactions_df, on=['date', 'store_nbr'], how='inner')

# Merge merged_df2 with holidays_events_df based on 'date' column
merged_df3 = merged_df2.merge(holiday_df, on='date', how='inner')

# Merge merged_df3 with oil_df based on 'date' column
merged_df = merged_df3.merge(oil_df, on='date', how='inner')

# View the first five rows of the merged dataset
merged_df.head()

The use of an inner merge in this time series forecasting project for corporation Favorita helps to ensure data consistency, avoid missing values, and focus on the relevant data for accurate predictions.

An inner merge type retains only the rows with matching values in the specified columns. In the context of time series forecasting, it allows us to merge datasets based on a common time index or timestamp. By performing an inner merge, we ensure that only the rows with corresponding timestamps in both datasets are included in the merged result. This is important for time series forecasting because you want to align the data from different sources based on their timestamps to build a consistent and accurate forecasting model.

With an inner merge, you eliminate any non-matching timestamps, which may not be useful for forecasting and could introduce inconsistencies in the data. By focusing on the intersection of the datasets, we can create a merged dataset that contains the necessary information for accurate time series forecasting.

checking the details of merged dataset


In [ ]:
details(merged_df,"merged_df")


The merged dataset after merging the train dataset with additional datasets contains 322,047 rows and 16 columns. Two columns have been renamed as a result of the merging, type_x and type_y.

checking for the details of the newly created columns


In [ ]:
# Check the unique values of the two unknown columns
print("Unique values of 'type_x':")
print(merged_df['type_x'].unique())
print()
print("Unique values of 'type_y':")
print(merged_df['type_y'].unique())

Unique values of 'type_x':
['D' 'E' 'C' 'A' 'B']

Unique values of 'type_y':

['Holiday' 'Additional' 'Transfer' 'Event' 'Bridge']

The merged dataset consists of 322,047 non-null observations. Two columns have been renamed as a result of the merging, type_x and type_y.

The type_x column represents the store type.

The type_y column represents the holiday type.

In [ ]:
merged_df = merged_df.rename(columns={"type_x": "store_type", "type_y": "holiday_type"})
merged_df.head()

checking details of merged dataset


In [ ]:
details(merged_df,"merged_df")

**SAVING THE MERGED DATASET**

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/data/Visualization_Data.csv', index=False)
merged_df.head()

**Feature Engineering**


create a copy of the merged dataset

In [ ]:
mergeddf_copy=merged_df.copy()
mergeddf_copy.head()
details(mergeddf_copy,'merged_df')

Extracting date components for getting sesonal, yearly and weekly trends

In [ ]:
mergeddf_copy['year'] = mergeddf_copy['date'].dt.year
mergeddf_copy['month'] = mergeddf_copy['date'].dt.month
mergeddf_copy['day'] = mergeddf_copy['date'].dt.day
mergeddf_copy.head()

droping columns which are not required for forecasting

In [ ]:
columns_to_drop = ['date','id', 'locale', 'locale_name', 'description', 'store_type', 'transferred', 'state']
mergeddf_copy = mergeddf_copy.drop(columns=columns_to_drop)

mergeddf_copy.head()

Classification of items to different product families

In [ ]:
unique_families = mergeddf_copy['family'].unique()
unique_families

In [ ]:

food_families = ['BEVERAGES', 'BREAD/BAKERY', 'FROZEN FOODS', 'MEATS', 'PREPARED FOODS', 'DELI','PRODUCE', 'DAIRY','POULTRY','EGGS','SEAFOOD']
home_families = ['HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES']
clothing_families = ['LINGERIE', 'LADIESWEAR']
grocery_families = ['GROCERY I', 'GROCERY II']
stationery_families = ['BOOKS', 'MAGAZINES','SCHOOL AND OFFICE SUPPLIES']
cleaning_families = ['HOME CARE', 'BABY CARE','PERSONAL CARE']
hardware_families = ['PLAYERS AND ELECTRONICS','HARDWARE']


mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(food_families), 'FOODS', mergeddf_copy['family'])
mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(home_families), 'HOME', mergeddf_copy['family'])
mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(clothing_families), 'CLOTHING', mergeddf_copy['family'])
mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(grocery_families), 'GROCERY', mergeddf_copy['family'])
mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(stationery_families), 'STATIONERY', mergeddf_copy['family'])
mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(cleaning_families), 'CLEANING', mergeddf_copy['family'])
mergeddf_copy['family'] = np.where(mergeddf_copy['family'].isin(hardware_families), 'HARDWARE', mergeddf_copy['family'])


mergeddf_copy.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram of Sales
plt.figure(figsize=(10,6))
plt.hist(merged_df['sales'], bins=20, color="steelblue", edgecolor="black")
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.title('Distribution of Sales')
plt.show()

# Boxplot of Sales
plt.figure(figsize=(10,4))
plt.boxplot(merged_df['sales'])
plt.ylabel('Sales')
plt.title('Boxplot of Sales')
plt.show()



distribution of the 'transactions' variable:

In [ ]:
# Histogram of Transactions
plt.figure(figsize=(10,6))
plt.hist(merged_df['transactions'], bins=20, color="steelblue", edgecolor="black")
plt.xlabel('Transactions')
plt.ylabel('Frequency')
plt.title('Distribution of Transactions')
plt.show()

# Boxplot of Transactions
plt.figure(figsize=(10,4))
plt.boxplot(merged_df['transactions'])
plt.ylabel('Transactions')
plt.title('Boxplot of Transactions')
plt.show()


Distribution of the 'Daily Oil Price' variable:

In [ ]:
# Histogram of Oil Price
plt.figure(figsize=(10,6))
plt.hist(merged_df['dcoilwtico'].dropna(), bins=20, color="teal", edgecolor="black")
plt.xlabel('Oil Price')
plt.ylabel('Frequency')
plt.title('Distribution of Oil Price')
plt.show()

# Boxplot of Oil Price
plt.figure(figsize=(10,4))
plt.boxplot(merged_df['dcoilwtico'].dropna())
plt.ylabel('Oil Price')
plt.title('Boxplot of Oil Price')
plt.show()



Trend of sales over time.

In [ ]:
# Group the data by date and calculate the total sales
daily_sales = merged_df.groupby('date')['sales'].sum().reset_index()

# Create a time series plot with slider
fig = px.line(daily_sales, x='date', y='sales')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title='Trend of Sales Over Time', title_x=0.5)
fig.show()

Trend of Daily Crude oil Prices Over Time

In [ ]:
# Visualizing the 'dcoilwtico' column to confirm if the trend is consistent.
fig = px.line(oil_df, x='date', y='dcoilwtico')
fig.update_layout(title='Trend of Oil Prices Over Time', title_x=0.5, xaxis_title='Date', yaxis_title='Oil Price')
fig.show()

Total Count of Sales by Store Type

In [ ]:
# Set the color palette to "viridis"
sns.set_palette("viridis")
# Calculate the total count and total sales per store type
store_type_counts = merged_df['store_type'].value_counts()
store_type_sales = merged_df.groupby('store_type')['sales'].sum()

# Create a bar plot with "viridis" color palette for total count
plt.figure(figsize=(8, 6))
sns.barplot(x=store_type_counts.index, y=store_type_counts.values)
plt.xlabel('Store Type')
plt.ylabel('Count')
plt.title('Total Count by Store Type')
plt.show()



Total Amount in Sales by Store Type

In [ ]:
# Order the store types by total sales
store_type_sales = store_type_sales.sort_values(ascending=False)

# Create a bar plot with "viridis" color palette for total sales
plt.figure(figsize=(8, 8))
sns.barplot(x=store_type_sales.index, y=store_type_sales.values, order=store_type_sales.index, palette="viridis")
plt.xlabel('Store Type')
plt.ylabel('Total Sales')
plt.title('Total Sales by Store Type')
plt.show()

Average Sales by City

In [ ]:
import numpy as np
import matplotlib.cm as cm

# Group by city and calculate the average sales
average_sales_by_city = merged_df.groupby('city')['sales'].mean()

# Sort the data by average sales in ascending order
average_sales_by_city = average_sales_by_city.sort_values(ascending=True)

# Define colors for the bar plot using 'viridis' color palette
colors = cm.viridis(np.linspace(0, 1, len(average_sales_by_city)))

# Plot the average sales by city horizontally
plt.figure(figsize=(8, 8))
plt.barh(average_sales_by_city.index, average_sales_by_city.values, color=colors)
plt.xlabel('Average Sales')
plt.ylabel('City')
plt.title('Average Sales by City')
plt.show()

Average Sales by State

In [ ]:
# Group by state and calculate the average sales
average_sales_by_state = merged_df.groupby('state')['sales'].mean()

# Sort the data by average sales in descending order
average_sales_by_state = average_sales_by_state.sort_values(ascending=True)

# Plot the average sales by state
plt.figure(figsize=(8, 8))
plt.barh(average_sales_by_state.index, average_sales_by_state.values, color=colors)
plt.xlabel('Average Sales')
plt.ylabel('State')
plt.title('Average Sales by State')
plt.xticks(rotation=45)
plt.show()

Relationship between sales and transactions.

In [ ]:
sns.scatterplot(x='transactions', y='sales', data=merged_df)
plt.xlabel('Transactions')
plt.ylabel('Sales')
plt.title('Relationship between Sales and Transactions')
plt.show()

Correlation matrix of numerical variables

In [ ]:
# Select numerical variables for correlation analysis
numerical_vars = ['sales', 'transactions', 'dcoilwtico']

# Compute correlation matrix
corr_matrix = merged_df[numerical_vars].corr()

# Plot heatmap
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

b. Scatter Plot Marrix of numerical Variables


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
# Select numerical variables for correlation analysis
numerical_vars = ['sales', 'transactions', 'dcoilwtico']

# Plot scatter plot matrix
sns.pairplot(merged_df[numerical_vars])
plt.show()

Stationarity Test

Stationarity implies that the statistical properties of the time series, such as mean and variance, remain constant over time. In this case, the ADF test was conducted on the 'sales' data from the 'merged_df' dataset. To perform the stationarity test, we will use the Augmented Dickey-Fuller (ADF) test commonly used to check for stationarity in a time series.

Null hypothesis (H0): The sales data is non-stationary.
Alternative hypothesis (H1): The sales data is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller
# Statistical Test of the 'sales' column in the merged_df using Adfuller
sales_data = merged_df['sales']

# Perform ADF test
result = adfuller(sales_data)

# Extract the test statistics and p-value from the result
test_statistic = result[0]
p_value = result[1]
critical_values = result[4]

# Print the test statistics and critical values
print(f"ADF Test Statistics: {test_statistic}")
print(f"P-value: {p_value}")
print("Critical Values:")
for key, value in critical_values.items():
    print(f"   {key}: {value}")

# Check the p-value against a significance level (e.g., 0.05)
if p_value <= 0.05:
    print("Reject the null hypothesis: The sales data is stationary.")
else:
    print("Fail to reject the null hypothesis: The sales data is non-stationary.")

Hypothesis Testing and Answering Key Analytical Questions

Hypothesis Testing
Null Hypothesis (H0): The promotional activities have a significant impact on store sales for Corporation Favorita.

Alternative Hypothesis (H1): The promotional activities have a significant impact on store sales for Corporation Favorita.

In [ ]:
# Extract the relevant variables for the hypothesis test
promo_sales = merged_df[merged_df['onpromotion'] == 1]['sales']
non_promo_sales = merged_df[merged_df['onpromotion'] == 0]['sales']

# Perform a two-sample t-test to compare sales between promotional and non-promotional periods
t_statistic, p_value = ttest_ind(promo_sales, non_promo_sales)


from scipy.stats import ttest_ind

# Extract the relevant variables for the hypothesis test
promo_sales = merged_df[merged_df['onpromotion'] == 1]['sales']
non_promo_sales = merged_df[merged_df['onpromotion'] == 0]['sales']

# Perform a two-sample t-test to compare sales between promotional and non-promotional periods
t_statistic, p_value = ttest_ind(promo_sales, non_promo_sales)

# Print the test result
print("Hypothesis Testing for Promotional Activities:")
print("Null Hypothesis (H0): The promotional activities do not have a significant impact on store sales.")
print("Alternative Hypothesis (H1): The promotional activities have a significant impact on store sales.")
print("-" * 50)
print("Test Statistic:", t_statistic)
print("P-value:", p_value)
print("=" * 50)
if p_value < 0.05:
    print("Reject the null hypothesis. Promotional activities have a significant impact on store sales at Corporation Favorita.")
else:
    print("Fail to reject the null hypothesis. Promotional activities do not have a significant impact on store sales at Corporation Favorita.")

1. Is the train dataset complete (has all the required dates)?

In [ ]:
# Check the completeness of the train dataset
min_date = train_df['date'].min()
max_date = train_df['date'].max()
expected_dates = pd.date_range(start=min_date, end=max_date)

missing_dates = expected_dates[~expected_dates.isin(train_df['date'])]

if len(missing_dates) == 0:
    print("The train dataset is complete. It includes all the required dates.")
else:
    print("The train dataset is incomplete. The following dates are missing:")
    print(missing_dates)

2. Which dates have the lowest and highest sales for each year?

In [ ]:
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year

lowest_sales_dates = merged_df.groupby('year')['date'].min()
highest_sales_dates = merged_df.groupby('year')['date'].max()

print("Dates with the lowest sales for each year:\n", lowest_sales_dates)
print("="*50)
print("Dates with the highest sales for each year:\n", highest_sales_dates)

3. Analyze the impact of the earthquake on sales

In [ ]:
# Assign the variable earthquake_date to the date the earthquake took place (April 16, 2016)
earthquake_date = pd.to_datetime('2016-04-16')

# Filter the sales data before and after the earthquake
sales_before_earthquake = train_df[train_df['date'] < earthquake_date]['sales']
sales_after_earthquake = train_df[train_df['date'] > earthquake_date]['sales']

# Set the colormap to viridis
colormap = cm.get_cmap('viridis')

# Plot the sales before and after the earthquake
plt.plot(sales_before_earthquake, color=colormap(0.2), label='Sales Before Earthquake')
plt.plot(sales_after_earthquake, color=colormap(0.8), label='Sales After Earthquake')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.show()

4. Determine if certain groups of stores sell more products

In [ ]:
# Group by cluster and calculate the average sales
average_sales_by_cluster = merged_df.groupby('cluster')['sales'].mean()

# Group by city and calculate the average sales
average_sales_by_city = merged_df.groupby('city')['sales'].mean()

# Group by state and calculate the average sales
average_sales_by_state = merged_df.groupby('state')['sales'].mean()

# Group by store type and calculate the average sales
average_sales_by_store_type = merged_df.groupby('store_type')['sales'].mean()

# Set the number of bars in each plot
num_bars = len(average_sales_by_cluster)

# Generate the colors using the viridis palette
colors = plt.cm.viridis(np.linspace(0, 1, num_bars))
# Sort the data by average sales in descending order
average_sales_by_cluster = average_sales_by_cluster.sort_values(ascending=False)

# Plot the average sales by cluster
plt.figure(figsize=(10, 8))
plt.bar(average_sales_by_cluster.index, average_sales_by_cluster.values, color=colors)
plt.xlabel('Cluster')
plt.ylabel('Average Sales')
plt.title('Average Sales by Cluster')

# Set the x-tick labels as integers
plt.xticks(range(1, len(average_sales_by_cluster.index) + 1))
plt.show()

In [ ]:
# Sort the data by average sales in descending order
average_sales_by_city = average_sales_by_city.sort_values(ascending=True)

# Plot the average sales by city horizontally
plt.figure(figsize=(8, 8))
plt.barh(average_sales_by_city.index, average_sales_by_city.values, color=colors)
plt.xlabel('Average Sales')
plt.ylabel('City')
plt.title('Average Sales by City')
plt.show()

In [ ]:
# Sort the data by average sales in descending order
average_sales_by_state = average_sales_by_state.sort_values(ascending=True)

# Plot the average sales by state
plt.figure(figsize=(8, 6))
plt.barh(average_sales_by_state.index, average_sales_by_state.values, color=colors)
plt.xlabel('State')
plt.ylabel('Average Sales')
plt.title('Average Sales by State')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plot the average sales by store type
plt.bar(average_sales_by_store_type.index, average_sales_by_store_type.values, color=colors)
plt.xlabel('Store Type')
plt.ylabel('Average Sales')
plt.title('Average Sales by Store Type')
plt.show()

5. Are sales affected by promotions, oil prices and holidays?

In [ ]:
# Calculate correlations between sales and promotions, oil prices, holidays
corr_sales_promotions = merged_df['sales'].corr(merged_df['onpromotion'])
corr_sales_oil = merged_df['sales'].corr(merged_df['dcoilwtico'])
corr_sales_holidays = merged_df['sales'].corr(merged_df['holiday_type'] == 'Holiday')

# Print the correlation values
print(f"Correlation between Sales and Promotions: {corr_sales_promotions}")
print(f"Correlation between Sales and Oil Prices: {corr_sales_oil}")
print(f"Correlation between Sales and Holidays: {corr_sales_holidays}")

Promotions: There is a positive correlation of approximately 0.42 between sales and promotions. This suggests that promotions have a moderate positive impact on sales. When promotions are running, there is an increased likelihood of higher sales.


Oil Prices: There is a weak negative correlation of approximately -0.06 between sales and oil prices. This indicates that there is a slight negative relationship between sales and oil prices. However, the correlation is close to zero, suggesting that oil prices have minimal impact on sales.


Holidays: There is a very weak negative correlation of approximately -0.04 between sales and holidays. This indicates that there is almost no relationship between sales and holidays. Holidays do not seem to have a significant impact on sales. These insights suggest that promotions have a relatively stronger influence on sales compared to oil prices and holidays. While promotions positively impact sales, oil prices and holidays show minimal or no relationship with sales.

6. What analysis can we get from the date and its extractable features?

In [ ]:
# Extracting Date Components
merged_df_copy['date'] = pd.to_datetime(merged_df_copy['date'])
merged_df_copy['year'] = merged_df_copy['date'].dt.year
merged_df_copy['month'] = merged_df_copy['date'].dt.month
merged_df_copy['day'] = merged_df_copy['date'].dt.day
merged_df_copy.head()

In [ ]:
# Set distinct colors for each year
colors = sns.color_palette("husl", n_colors=len(merged_df_copy['year'].unique()))

# Visualize the Monthly Sales Trend
monthly_sales = merged_df_copy.groupby(['year', 'month'])['sales'].sum().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_sales, x='month', y='sales', hue='year', palette=colors)
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(range(1, 13), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Year')
plt.show()

In [ ]:
# Visualize the Day of the Week Sales Pattern
merged_df_copy['day_of_week'] = merged_df_copy['date'].dt.dayofweek
day_of_week_sales = merged_df_copy.groupby('day_of_week')['sales'].mean().reset_index()
plt.figure(figsize=(8, 6))
sns.barplot(data=day_of_week_sales, x='day_of_week', y='sales')
plt.title('Average Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Sales')
plt.xticks(range(7), labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.show()

In [ ]:
# Visualize the Yearly Sales Trend
yearly_sales = merged_df_copy.groupby('year')['sales'].sum().reset_index()
plt.figure(figsize=(8, 6))
sns.barplot(data=yearly_sales, x='year', y='sales')
plt.title('Yearly Sales Trend')
plt.xlabel('Year')
plt.ylabel('Total Sales')
plt.show()

7. Are there certain product families types that exhibit higher sales performance?

In [ ]:
# Group the data by product family and calculate the total sales
family_sales = merged_df.groupby('family')['sales'].sum().sort_values(ascending=False)

# Select the top 10 product families
top_10_families = family_sales.head(10)

# Plot the relationship between product family and sales for the top 10 families
plt.figure(figsize=(12, 6))
sns.barplot(x=top_10_families.index, y=top_10_families.values, palette='viridis')
plt.xlabel('Product Family')
plt.ylabel('Total Sales')
plt.title('Total Sales by Product Family (Top 10)')
plt.xticks(rotation=45, ha='right')
plt.show()

8. How does the sales trend vary across different store numbers?

In [ ]:
store_sales = merged_df.groupby('store_nbr')['sales'].sum()

plt.figure(figsize=(14, 6))
plt.bar(store_sales.index, store_sales.values, color=cm.viridis(store_sales.values/max(store_sales.values)))
plt.xlabel('Store Number')
plt.ylabel('Total Sales')
plt.title('Sales by Store Number')

# Set the X-axis limits and ticks
plt.xlim(0, 54)
plt.xticks(range(55))

plt.show()

**FEATURE SCALING**

Standard scaler is used to scale the dataset and make mean 0 and standard deviation 1

In [ ]:
# Scaling Numeric Variables (Min-Max Scaling)
# create an instance of StandardScaler
scaler = StandardScaler()

num_cols = ['sales', 'transactions', 'dcoilwtico']

# fit and transform the numerical columns
mergeddf_copy[num_cols] = scaler.fit_transform(mergeddf_copy[num_cols])

# Display the updated dataframe
mergeddf_copy.head()

Encoding the categorical columns to one hot encoded vector

In [ ]:

# Define the categorical columns to encode
categorical_columns = ["family", "city", "holiday_type"]

# Perform one-hot encoding
encoder = OneHotEncoder()
one_hot_encoded_data = encoder.fit_transform(mergeddf_copy[categorical_columns])

# Create column names for the one-hot encoded data
column_names = encoder.get_feature_names_out(categorical_columns)

# Convert the one-hot encoded data to a DataFrame
merged_df_encoded = pd.DataFrame(one_hot_encoded_data.toarray(), columns=column_names)

# Concatenate the original dataframe with the one-hot encoded data
merged_df_encoded = pd.concat([mergeddf_copy, merged_df_encoded], axis=1)

# Drop the original categorical columns
merged_df_encoded.drop(categorical_columns, axis=1, inplace=True)

# Print the head of the encoded DataFrame
merged_df_encoded.head()

**TEST DATASET test_df**

In [ ]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df.head()

Drop irrelevant columns

In [ ]:
columns_to_drop = ['date', 'id']
test_df = test_df.drop(columns=columns_to_drop)
test_df.head()

In [ ]:
food_families = ['BEVERAGES', 'BREAD/BAKERY', 'FROZEN FOODS', 'MEATS', 'PREPARED FOODS', 'DELI','PRODUCE', 'DAIRY','POULTRY','EGGS','SEAFOOD']
home_families = ['HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES']
clothing_families = ['LINGERIE', 'LADIESWEAR']
grocery_families = ['GROCERY I', 'GROCERY II']
stationery_families = ['BOOKS', 'MAGAZINES','SCHOOL AND OFFICE SUPPLIES']
cleaning_families = ['HOME CARE', 'BABY CARE','PERSONAL CARE']
hardware_families = ['PLAYERS AND ELECTRONICS','HARDWARE']

test_df['family'] = np.where(test_df['family'].isin(food_families), 'FOODS', test_df['family'])
test_df['family'] = np.where(test_df['family'].isin(home_families), 'HOME', test_df['family'])
test_df['family'] = np.where(test_df['family'].isin(clothing_families), 'CLOTHING', test_df['family'])
test_df['family'] = np.where(test_df['family'].isin(grocery_families), 'GROCERY', test_df['family'])
test_df['family'] = np.where(test_df['family'].isin(stationery_families), 'STATIONERY', test_df['family'])
test_df['family'] = np.where(test_df['family'].isin(cleaning_families), 'CLEANING', test_df['family'])
test_df['family'] = np.where(test_df['family'].isin(hardware_families), 'HARDWARE', test_df['family'])

ENCODE CATEGORICAL COLUMNS

In [ ]:
categorical_columns = ["family"]

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder()

# Perform one-hot encoding on the 'test_df' data for the specified categorical columns
one_hot_encoded_data = encoder.fit_transform(test_df[categorical_columns])

# Get the column names for the one-hot encoded data
column_names = encoder.get_feature_names_out(categorical_columns)

# Create a DataFrame with the one-hot encoded data and corresponding column names
test_df_encoded = pd.DataFrame(one_hot_encoded_data.toarray(), columns=column_names)

# Concatenate the original 'test_df' with the one-hot encoded data
test_df_encoded = pd.concat([test_df, test_df_encoded], axis=1)

# Drop the original categorical columns since they have been encoded
test_df_encoded.drop(categorical_columns, axis=1, inplace=True)

# Display the updated 'test_df_encoded' DataFrame
test_df_encoded.head()

**MODELLING**

In [ ]:
train_set = merged_df_encoded.loc[merged_df_encoded['year'].isin([2013, 2014, 2015, 2016])]
eval_set = merged_df_encoded.loc[merged_df_encoded['year'] == 2017]


In [ ]:
train_set.shape

In [ ]:
eval_set.head()

In [ ]:
# Separate the target variable and features for training and testing
X_train = train_set.drop('sales', axis=1)
y_train = train_set['sales']


X_eval = eval_set.drop('sales', axis=1)
y_eval = eval_set['sales']

In [ ]:
results_df = pd.DataFrame(columns=['Model', 'RMSLE', 'RMSE', 'MSE', 'MAE'])

**MODEL TRAINING**

**LINEAR REGRESSION**

Linear Regression is a predictive modeling technique used to find a linear relationship between a dependent variable (the one you want to predict) and one or more independent variables (the features you use for prediction).

In [ ]:
# Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_eval)

# Calculate metrics
lr_mse = mean_squared_error(y_eval, lr_predictions)
lr_mae = mean_absolute_error(y_eval, lr_predictions)

# Apply the absolute value function to both y_eval and lr_predictions
y_eval_abs = abs(y_eval)
lr_predictions_abs = abs(lr_predictions)

# Calculate the Root Mean Squared Logarithmic Error (RMSLE)
lr_rmsle = np.sqrt(mean_squared_log_error(y_eval_abs, lr_predictions_abs))

# Create a DataFrame to store results for Linear Regression
results_lr = pd.DataFrame({'Model': ['Linear Regression'],
                            'RMSLE': [lr_rmsle],
                            'RMSE': [np.sqrt(lr_mse)],
                            'MSE': [lr_mse],
                            'MAE': [lr_mae]}).round(2)

# Print the results_lr dataframe
results_lr

**RANDOM FOREST**

Random Forest is an ensemble learning method. Instead of relying on a single complex model, it builds a large number of individual Decision Trees and then combines their predictions to make a final, more accurate prediction.

In [ ]:
# Random Forest Regression Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_eval)

# Calculate metrics
rf_mse = mean_squared_error(y_eval, rf_predictions)
rf_mae = mean_absolute_error(y_eval, rf_predictions)

# Apply the absolute value function to both y_eval and rf_predictions
y_eval_abs = abs(y_eval)
rf_predictions_abs = abs(rf_predictions)

# Calculate the Root Mean Squared Logarithmic Error (RMSLE)
rf_rmsle = np.sqrt(mean_squared_log_error(y_eval_abs, rf_predictions_abs))

# Create a DataFrame to store results for Random Forest
results_rf = pd.DataFrame({'Model': ['Random Forest'],
                            'RMSLE': [rf_rmsle],
                            'RMSE': [np.sqrt(rf_mse)],
                            'MSE': [rf_mse],
                            'MAE': [rf_mae]}).round(2)

# Print the results_rf dataframe
results_rf

**Gradient Boosting**

Gradient Boosting is an ensemble learning method that builds models sequentially.

Instead of growing many trees independently (like Random Forest), Gradient Boosting builds one tree at a time.

Each new tree tries to fix the errors made by the previous trees.

It minimizes a loss function (like MSE) using gradient descent, hence the name.

In [ ]:
# Gradient Boosting Regression Model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_predictions = gb_model.predict(X_eval)

# Calculate metrics
gb_mse = mean_squared_error(y_eval, gb_predictions)
gb_mae = mean_absolute_error(y_eval, gb_predictions)

# Apply the absolute value function to both y_eval and gb_predictions
y_eval_abs = abs(y_eval)
gb_predictions_abs = abs(gb_predictions)

# Calculate the Root Mean Squared Logarithmic Error (RMSLE)
gb_rmsle = np.sqrt(mean_squared_log_error(y_eval_abs, gb_predictions_abs))

# Create a DataFrame to store results for Gradient Boosting
results_gb = pd.DataFrame({'Model': ['Gradient Boosting'],
                            'RMSLE': [gb_rmsle],
                            'RMSE': [np.sqrt(gb_mse)],
                            'MSE': [gb_mse],
                            'MAE': [gb_mae]}).round(2)

# Print the results_gb dataframe
results_gb

**ARIMA**



ARIMA stands for AutoRegressive Integrated Moving Average.

It’s a time series forecasting model that combines three components:

AR (AutoRegressive, p): relationship between current value and its past values.

I (Integrated, d): differencing to make the series stationary.

MA (Moving Average, q): relationship between current value and past forecast errors.

Best for univariate time series forecasting.

In [ ]:
# d and q are equal to zero as data is already stationary
p = 1
d = 0
q = 0

# Create an instance of the ARIMA model
arima_model = ARIMA(y_train, order=(p, d, q))

# Fit the model to the training data
arima_model_fit = arima_model.fit()

# Make predictions on the evaluation data
arima_predictions = arima_model_fit.predict(start=len(y_train), end=len(y_train) + len(X_eval) - 1)

# Calculate evaluation metrics
arima_mse = mean_squared_error(y_eval, arima_predictions)
arima_rmse = np.sqrt(arima_mse)

# Apply the absolute value function to y_eval to remove negative signs
y_eval_abs = abs(y_eval)
arima_predictions_abs = abs(arima_predictions)

# Calculate the Mean Absolute Error (MAE)
arima_mae = mean_absolute_error(y_eval, arima_predictions)

# Calculate the Root Mean Squared Logarithmic Error (RMSLE)
arima_rmsle = np.sqrt(mean_squared_log_error(y_eval_abs, arima_predictions_abs))

# Create a DataFrame to store results for ARIMA
results_arima = pd.DataFrame({'Model': ['ARIMA'],
                            'RMSLE': [arima_rmsle],
                            'RMSE': [np.sqrt(arima_mse)],
                            'MSE': [arima_mse],
                            'MAE': [arima_mae]}).round(2)

# Print the results_arima dataframe
results_arima

**SARIMA**


SARIMA stands for Seasonal AutoRegressive Integrated Moving Average.

It is an extension of ARIMA that can handle seasonality in time series.

Components:

(p, d, q): same as ARIMA → autoregression, differencing, moving average.

(P, D, Q, s): seasonal part → captures repeating seasonal patterns.

P: seasonal autoregressive order.

D: seasonal differencing order.

Q: seasonal moving average order.

In [ ]:
# Set the order and seasonal order parameters
# Seasonal autoregressive order
P = 0
# Seasonal differencing order
D = 0
# Seasonal moving average order
Q = 0
# Number of time steps in each season (chosen based on the number of months each year)
s = 12

# Create an instance of the SARIMA model
sarima_model = SARIMAX(endog=y_train, exog=X_train, order=(p, d, q), seasonal_order=(P, D, Q, s))

# Fit the model to the training data
sarima_fit = sarima_model.fit()

# Make predictions on the evaluation data
sarima_predictions = sarima_fit.forecast(steps=len(y_eval), exog=X_eval)

# Calculate metrics
sarima_mse = mean_squared_error(y_eval, sarima_predictions)
sarima_rmse = np.sqrt(sarima_mse)
sarima_mae = mean_absolute_error(y_eval, sarima_predictions)
sarima_rmsle = np.sqrt(mean_squared_error(np.log1p(y_eval), np.log1p(sarima_predictions)))

# Create a DataFrame to store results for SARIMA
results_sarima = pd.DataFrame({'Model': ['SARIMA'],
                                'RMSLE': [sarima_rmsle],
                                'RMSE': [sarima_rmse],
                                'MSE': [sarima_mse],
                                'MAE': [sarima_mae]}).round(2)

# Print the results_sarima dataframe
results_sarima

**XGBOOST**

In [ ]:


# Create the model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators =100, learning_rate=0.1)

# Fit the model
xg_reg.fit(X_train, y_train)

# Predict on training set (or on test set if available)
y_pred = xg_reg.predict(X_train)

# Evaluate
xgboost_mse = mean_squared_error(y_eval, sarima_predictions)
xgboost_rmse = np.sqrt(sarima_mse)
xgboost_mae = mean_absolute_error(y_eval, sarima_predictions)
xgboost_rmsle = np.sqrt(mean_squared_error(np.log1p(y_eval), np.log1p(sarima_predictions)))

results_xgboost = pd.DataFrame({'Model': ['XGBOOST'],
                                'RMSLE': [xgboost_rmsle],
                                'RMSE': [xgboost_rmse],
                                'MSE': [xgboost_mse],
                                'MAE': [xgboost_mae]}).round(2)
results_xgboost


In [ ]:
# Append all results to the results dataframe
results_df = pd.concat([results_df, results_lr], ignore_index=True)
results_df = pd.concat([results_df, results_rf], ignore_index=True)
results_df = pd.concat([results_df, results_gb], ignore_index=True)
results_df = pd.concat([results_df, results_arima], ignore_index=True)
results_df = pd.concat([results_df, results_sarima], ignore_index=True)
results_df = pd.concat([results_df, results_xgboost],ignore_index=True)

# Sort the results_df based on RMSE in ascending order
results_df = results_df.sort_values(by='RMSLE', ascending=True)

# Reset the index of the DataFrame
results_df = results_df.reset_index(drop=True)

# Print the Final Results dataframe
results_df.drop_duplicates()

The RMSLE is a metric used when the target variable has a wide range of values. A lower RMSLE indicates a better model fit.
The RMSE measures the average magnitude of the errors between predicted and actual values. A lower RMSE indicates better model performance.
The MSE is the average of the squared errors and provides a measure of the model's overall accuracy. A lower MSE indicates better performance.
The MAE measures the average magnitude of the errors without considering their direction. A lower MAE indicates better model accuracy.
The RMSLE and RMSE represent the model's prediction error, with lower values indicating better performance. The MSE provides a measure of the squared error between predicted and actual values, while the MAE measures the average absolute difference between predictions and actuals.

These results indicate the performance of each model in terms of different metrics. Lower values of RMSLE, RMSE, MSE, and MAE indicate better model performance. Based on these results, the Random Forest and Gradient Boosting models appear to perform better than the Linear Regression, ARIMA, and SARIMA models in terms of the provided metrics.


Best performing model overall:  Random Forest has the lowest RMSLE (0.23) and lowest MAE (0.25), indicating better accuracy and robustness to outliers.

Gradient Boosting shows slightly higher RMSLE (0.24) but lower RMSE (0.67) and MSE (0.45), meaning it's good in terms of minimizing large errors but slightly worse than Random Forest in relative error.

Linear Regression and SARIMA perform similarly, with higher errors compared to ensemble models.

ARIMA performs the worst across all metrics, indicating it's less suitable for this dataset or problem.

**HYPERPARAMETER TUNING**

In [ ]:
# Define the parameter grid for tuning the random forest model
param_grid_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['sqrt', 'log2', 0.5]
}

# Create Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV
random_search_rf = RandomizedSearchCV(rf_model, param_distributions=param_grid_rf,
                                      n_iter=10, scoring='neg_mean_squared_error', cv=5,
                                      n_jobs=-1, random_state=42)

# Fit RandomizedSearchCV to the data
random_search_rf.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_rf_model = random_search_rf.best_estimator_
best_rf_params = random_search_rf.best_params_

# Make predictions using the best model
best_rf_predictions = best_rf_model.predict(X_eval)

# Calculate metrics for the best model
best_rf_mse = mean_squared_error(y_eval, best_rf_predictions)
best_rf_rmse = np.sqrt(best_rf_mse)
best_rf_mae = mean_absolute_error(y_eval, best_rf_predictions)

# Apply absolute value to both predicted and target values
abs_best_rf_predictions = np.abs(best_rf_predictions)
abs_y_eval = np.abs(y_eval)

# Calculate RMSLE using the absolute values
best_rf_rmsle = np.sqrt(mean_squared_log_error(abs_y_eval, abs_best_rf_predictions))

# Create a DataFrame to store results for the best Random Forest model
best_results_rf = pd.DataFrame({'Model': ['Best Random Forest'],
                                'RMSLE': [best_rf_rmsle],
                                'RMSE': [best_rf_rmse],
                                'MSE': [best_rf_mse],
                                'MAE': [best_rf_mae]}).round(2)

# Print the best_results_rf dataframe
best_results_rf

In [ ]:
# Print the best parameters
print("Best Parameters for Random Forest Model:")
print(best_rf_params)

The Random Forest model was tuned using a randomized search to find the best combination of hyperparameters that optimize its performance. The search yielded the following set of optimal hyperparameters:

Number of Estimators (Trees): 500
Minimum Samples Split: 2
Minimum Samples Leaf: 1
Maximum Features: 0.5
Maximum Depth: 10



These hyperparameters represent the configuration that resulted in the best performance for the Random Forest model on the evaluation data. The hyperparameters were selected based on their impact on the model's ability to minimize the Mean Squared Error (MSE), which is a common measure of predictive accuracy. By fine-tuning these hyperparameters, the model's ability to generalize to unseen data has been significantly improved.

After applying these optimal hyperparameters, the Random Forest model's performance on the evaluation data is as follows:

Root Mean Squared Logarithmic Error (RMSLE): 0.21 After tuning the Random Forest model, the RMSLE improved from 0.22 to 0.21, suggesting that the model's ability to handle the variation in target values has improved.

Root Mean Squared Error (RMSE): 0.59 After tuning, the RMSE decreased from 0.71 to 0.59, indicating that the model's predictions are more accurate on average.

Mean Squared Error (MSE): 0.35 After tuning, the MSE decreased from 0.51 to 0.35, indicating that the model's predictions are closer to the actual values on average.

Mean Absolute Error (MAE): 0.23 After tuning, the MAE decreased from 0.24 to 0.23, suggesting that the model's predictions are closer to the true values on average.

The hyperparameter tuning process for the Random Forest model led to improvements in all evaluation metrics, indicating that the model's performance has been enhanced. This suggests that the tuned Random Forest model is a better fit for the data and is capable of making more accurate predictions on the target variable These performance metrics provide insights into the model's accuracy and how well it predicts sales values.

In summary, the Random Forest model, with its optimized hyperparameters, demonstrates improved predictive performance compared to its initial configuration. It effectively captures the relationships between features and sales, resulting in more accurate predictions on the evaluation dataset.

**SAVING THE BEST MODEL**

In [ ]:

key_components = {
    'model': best_rf_model,
    'best_params': best_rf_params,
    'best_score': best_rf_rmsle
}


with open('best_rf_model_components.pkl', 'wb') as file:
    pickle.dump(key_components, file)